In [2]:
# Debugging surface view factor on simple voxel scenes
# - Case A: Two buildings with clear line of sight (LoS)
# - Case B: Same buildings but occluded by a thick tree wall
#
# We report the mean view factor for east-facing faces of the first building.

import os, sys
import numpy as np

# Ensure the repo's src is on path when running from docs/
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(repo_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from voxcity.simulator.view import get_surface_view_factor

meshsize = 1.0

# Build simple scene
# Grid: X x Y x Z
NX, NY, NZ = 12, 8, 5

# Class IDs used in the library:
#  0 = air, -2 = trees, -3 = buildings

def build_scene(with_tree_wall=False, tree_thickness=4):
    vox = np.zeros((NX, NY, NZ), dtype=np.int32)
    # Building 1: block at x=2..3, y=3..4, z=0..2 (2x2x3 voxels)
    vox[2:4, 3:5, 0:3] = -3
    # Building 2: block at x=7..8, y=3..4, z=0..2
    vox[7:9, 3:5, 0:3] = -3
    if with_tree_wall:
        # Tree wall between them at x=4..(4+tree_thickness-1)
        x0 = 4
        x1 = min(NX, x0 + tree_thickness)
        vox[x0:x1, 3:5, 0:3] = -2
    return vox


def run_view_factor(voxel_data):
    mesh = get_surface_view_factor(
        voxel_data,
        meshsize,
        target_values=(-3,),           # buildings as targets
        inclusion_mode=True,           # count rays that hit buildings
        N_azimuth=36,
        N_elevation=9,
        progress_report=False,
    )
    return mesh


def summarize_east_faces(mesh):
    # East-facing faces of building 1 (normals ~ +X, centers x < 4.5)
    centers = mesh.triangles_center
    normals = mesh.face_normals
    vf = mesh.metadata["view_factor_values"]

    mask_east = (np.abs(normals[:, 0] - 1.0) < 1e-8) & (centers[:, 0] < 4.5)
    vals = vf[mask_east]
    if vals.size == 0:
        return {"count": 0, "mean": np.nan, "min": np.nan, "max": np.nan}
    vals = vals[~np.isnan(vals)]
    if vals.size == 0:
        return {"count": 0, "mean": np.nan, "min": np.nan, "max": np.nan}
    return {
        "count": int(vals.size),
        "mean": float(np.mean(vals)),
        "min": float(np.min(vals)),
        "max": float(np.max(vals)),
    }

# Case A: Clear LoS
vox_clear = build_scene(with_tree_wall=False)
mesh_clear = run_view_factor(vox_clear)
stats_clear = summarize_east_faces(mesh_clear)
print("Case A (clear LoS) — east faces of building 1:", stats_clear)

# Case B: Occluded by a thick tree wall (forces heavy attenuation)
vox_trees = build_scene(with_tree_wall=True, tree_thickness=6)
mesh_trees = run_view_factor(vox_trees)
stats_trees = summarize_east_faces(mesh_trees)
print("Case B (tree wall) — east faces of building 1:", stats_trees)

# Expected behavior after fix:
# - Case A: mean/max > 0 (some rays reach the other building)
# - Case B: mean/max ~ 0 (rays attenuate in trees and should NOT be counted as hits for buildings)


Case A (clear LoS) — east faces of building 1: {'count': 12, 'mean': 0.20806100217864923, 'min': 0.12418300653594772, 'max': 0.2908496732026144}
Case B (tree wall) — east faces of building 1: {'count': 12, 'mean': 0.0, 'min': 0.0, 'max': 0.0}


In [ ]:
# Landmark (-30) target test: buildings placed on ±X and ±Y relative to landmark
# Only faces oriented inward toward the landmark should have VF>0; outward faces should be ~0.

import numpy as np
from voxcity.simulator.view import get_surface_view_factor

meshsize = 1.0
NX, NY, NZ = 20, 20, 6

AIR = 0
TREE = -2
BUILDING = -3
LANDMARK = -30

# Scene: landmark pillar at center; four building pillars at +/-dx or +/-dy
cx, cy = 10, 10
lm_h = 3
b_h = 3

def build_landmark_scene(dx=5, dy=5, gap=2):
    vox = np.zeros((NX, NY, NZ), dtype=np.int32)
    # Landmark at center (1x1xlm_h)
    vox[cx:cx+1, cy:cy+1, 0:lm_h] = LANDMARK
    # West building: to the -X side, inward faces point +X
    vox[cx-dx-1:cx-dx, cy:cy+1, 0:b_h] = BUILDING
    # East building: to the +X side, inward faces point -X
    vox[cx+dx+0:cx+dx+1, cy:cy+1, 0:b_h] = BUILDING
    # South building: to the -Y side, inward faces point +Y
    vox[cx:cx+1, cy-dy-1:cy-dy, 0:b_h] = BUILDING
    # North building: to the +Y side, inward faces point -Y
    vox[cx:cx+1, cy+dy+0:cy+dy+1, 0:b_h] = BUILDING
    return vox

vox = build_landmark_scene(dx=4, dy=4)
mesh = get_surface_view_factor(
    vox,
    meshsize,
    target_values=(LANDMARK,),
    inclusion_mode=True,
    N_azimuth=72,
    N_elevation=12,
    progress_report=False,
)

centers = mesh.triangles_center
normals = mesh.face_normals
vf = mesh.metadata["view_factor_values"]

# Categorize faces: west, east, south, north buildings via x/y center location
west_mask = centers[:, 0] < (cx - 3) * meshsize
east_mask = centers[:, 0] > (cx + 3) * meshsize
south_mask = centers[:, 1] < (cy - 3) * meshsize	north_mask = centers[:, 1] > (cy + 3) * meshsize

# Inward normals relative to landmark
inward_west = west_mask & (normals[:, 0] > +0.99)   # +X toward landmark
outward_west = west_mask & (normals[:, 0] < -0.99)  # -X away

inward_east = east_mask & (normals[:, 0] < -0.99)   # -X toward
outward_east = east_mask & (normals[:, 0] > +0.99)  # +X away

inward_south = south_mask & (normals[:, 1] > +0.99) # +Y toward
outward_south = south_mask & (normals[:, 1] < -0.99)# -Y away

inward_north = north_mask & (normals[:, 1] < -0.99) # -Y toward
outward_north = north_mask & (normals[:, 1] > +0.99)# +Y away

import math

def stats(mask):
    vals = vf[mask]
    vals = vals[~np.isnan(vals)]
    if vals.size == 0:
        return {"count": 0, "mean": math.nan, "max": math.nan}
    return {"count": int(vals.size), "mean": float(vals.mean()), "max": float(vals.max())}

print("West inward:", stats(inward_west))
print("West outward:", stats(outward_west))
print("East inward:", stats(inward_east))
print("East outward:", stats(outward_east))
print("South inward:", stats(inward_south))
print("South outward:", stats(outward_south))
print("North inward:", stats(inward_north))
print("North outward:", stats(outward_north))

# Expected: all inward categories > 0, all outward categories ~ 0.0


In [3]:
# Utilities: hemisphere generation, face selection, debug tracer
import math
from typing import Dict, List, Tuple

from voxcity.simulator.view import rotate_vector_axis_angle, compute_view_factor_for_all_faces

AZ_DEFAULT = 36
EL_DEFAULT = 9

def make_hemisphere_dirs(N_azimuth=AZ_DEFAULT, N_elevation=EL_DEFAULT):
    azimuth_angles = np.linspace(0, 2*np.pi, N_azimuth, endpoint=False)
    elevation_angles = np.linspace(0, np.pi/2, N_elevation)
    hemi = []
    for elev in elevation_angles:
        sin_e = np.sin(elev)
        cos_e = np.cos(elev)
        for az in azimuth_angles:
            x = cos_e * np.cos(az)
            y = cos_e * np.sin(az)
            z = sin_e
            hemi.append([x, y, z])
    return np.array(hemi, dtype=np.float64)


def face_orientation_masks(mesh, building1_x_threshold=4.5):
    centers = mesh.triangles_center
    normals = mesh.face_normals
    # Partition faces of building 1 by normal direction
    sel = centers[:, 0] < building1_x_threshold
    eps = 1e-6
    return {
        "+X": sel & (np.abs(normals[:, 0] - 1.0) < eps),
        "-X": sel & (np.abs(normals[:, 0] + 1.0) < eps),
        "+Y": sel & (np.abs(normals[:, 1] - 1.0) < eps),
        "-Y": sel & (np.abs(normals[:, 1] + 1.0) < eps),
        "+Z": sel & (np.abs(normals[:, 2] - 1.0) < eps),
        "-Z": sel & (np.abs(normals[:, 2] + 1.0) < eps),
    }


def summarize_by_orientation(mesh) -> Dict[str, Dict[str, float]]:
    vf = mesh.metadata["view_factor_values"]
    masks = face_orientation_masks(mesh)
    out = {}
    for name, m in masks.items():
        vals = vf[m]
        vals = vals[~np.isnan(vals)]
        if vals.size == 0:
            out[name] = {"count": 0, "mean": np.nan, "min": np.nan, "max": np.nan}
        else:
            out[name] = {
                "count": int(vals.size),
                "mean": float(np.mean(vals)),
                "min": float(np.min(vals)),
                "max": float(np.max(vals)),
            }
    return out


def print_orientation_summary(title: str, mesh):
    print(title)
    stats = summarize_by_orientation(mesh)
    for k, v in stats.items():
        print(f"  {k}: {v}")


# Pure-Python DDA tracer for detailed classification
# Returns one of: 'air_exit', 'tree_block', 'hit_building', 'hit_tree', 'hit_other', 'oob'

def trace_ray_debug(voxel_data: np.ndarray,
                    origin_vox: np.ndarray,
                    direction: np.ndarray,
                    meshsize: float,
                    tree_k: float = 0.6,
                    tree_lad: float = 1.0,
                    cutoff: float = 0.01) -> str:
    nx, ny, nz = voxel_data.shape
    x0, y0, z0 = origin_vox
    dx, dy, dz = direction.astype(np.float64)
    norm = math.sqrt(dx*dx + dy*dy + dz*dz)
    if norm == 0:
        return 'air_exit'
    dx /= norm; dy /= norm; dz /= norm
    x, y, z = x0 + 0.5, y0 + 0.5, z0 + 0.5
    i, j, k = int(x0), int(y0), int(z0)
    step_x = 1 if dx >= 0 else -1
    step_y = 1 if dy >= 0 else -1
    step_z = 1 if dz >= 0 else -1
    EPS = 1e-12
    if abs(dx) > EPS:
        t_max_x = ((i + (step_x > 0)) - x) / dx
        t_delta_x = abs(1.0 / dx)
    else:
        t_max_x = math.inf; t_delta_x = math.inf
    if abs(dy) > EPS:
        t_max_y = ((j + (step_y > 0)) - y) / dy
        t_delta_y = abs(1.0 / dy)
    else:
        t_max_y = math.inf; t_delta_y = math.inf
    if abs(dz) > EPS:
        t_max_z = ((k + (step_z > 0)) - z) / dz
        t_delta_z = abs(1.0 / dz)
    else:
        t_max_z = math.inf; t_delta_z = math.inf

    T = 1.0
    last_t = 0.0
    while 0 <= i < nx and 0 <= j < ny and 0 <= k < nz:
        val = voxel_data[i, j, k]
        t_next = min(t_max_x, t_max_y, t_max_z)
        seg_len = max(0.0, (t_next - last_t) * meshsize)
        if val == -2:  # tree
            T *= math.exp(-tree_k * tree_lad * seg_len)
            if T < cutoff:
                return 'tree_block'
        elif val == -3:
            return 'hit_building'
        elif val != 0:
            # other opaque
            return 'hit_other'

        last_t = t_next
        if t_max_x < t_max_y:
            if t_max_x < t_max_z:
                t_max_x += t_delta_x; i += step_x
            else:
                t_max_z += t_delta_z; k += step_z
        else:
            if t_max_y < t_max_z:
                t_max_y += t_delta_y; j += step_y
            else:
                t_max_z += t_delta_z; k += step_z
    return 'air_exit'


def face_directional_breakdown(mesh, voxel_data, meshsize, face_selector, N_azimuth=AZ_DEFAULT, N_elevation=EL_DEFAULT, offset_vox=0.51):
    centers = mesh.triangles_center
    normals = mesh.face_normals
    indices = np.where(face_selector)[0]
    if indices.size == 0:
        print("No faces selected.")
        return
    fidx = indices[0]
    center = centers[fidx]
    normal = normals[fidx]

    # Make hemisphere and rotate to face normal (same as production)
    hemi = make_hemisphere_dirs(N_azimuth, N_elevation)
    z_axis = np.array([0.0, 0.0, 1.0])
    nrm = np.linalg.norm(normal)
    dot = np.dot(z_axis, normal) / (nrm + 1e-15)
    dot = np.clip(dot, -1.0, 1.0)
    angle = math.acos(dot)
    if abs(dot - 1.0) < 1e-9:
        local_dirs = hemi
    elif abs(dot + 1.0) < 1e-9:
        axis_180 = np.array([1.0, 0.0, 0.0])
        local_dirs = np.array([rotate_vector_axis_angle(v, axis_180, math.pi) for v in hemi])
    else:
        rot_axis = np.cross(z_axis, normal)
        local_dirs = np.array([rotate_vector_axis_angle(v, rot_axis, angle) for v in hemi])

    # Outward and upward filter (same as production)
    outward = np.array([np.dot(d, normal) > 0.0 for d in local_dirs])
    upward = np.array([d[2] > 0.0 for d in local_dirs])
    valid = outward & upward
    dirs = local_dirs[valid]

    # Ray origin
    origin_vox = (center / meshsize) + (normal / (nrm + 1e-15)) * offset_vox

    # Classify results
    from collections import Counter
    counts = Counter()
    for d in dirs:
        outcome = trace_ray_debug(voxel_data, origin_vox, d, meshsize)
        counts[outcome] += 1

    total = len(dirs)
    print(f"Face idx {fidx}, normal {normal}, total valid rays: {total}")
    for k, v in counts.items():
        print(f"  {k}: {v} ({v/total:.3f})")
    return counts, total


In [4]:
# Orientation summaries for building->building visibility
mesh_clear = run_view_factor(vox_clear)
mesh_trees = run_view_factor(vox_trees)

print_orientation_summary("Case A (clear LoS)", mesh_clear)
print_orientation_summary("Case B (tree wall)", mesh_trees)


Case A (clear LoS)
  +X: {'count': 12, 'mean': 0.20806100217864923, 'min': 0.12418300653594772, 'max': 0.2908496732026144}
  -X: {'count': 12, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  +Y: {'count': 12, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  -Y: {'count': 12, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  +Z: {'count': 8, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  -Z: {'count': 8, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
Case B (tree wall)
  +X: {'count': 12, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  -X: {'count': 12, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  +Y: {'count': 12, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  -Y: {'count': 12, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  +Z: {'count': 8, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
  -Z: {'count': 8, 'mean': 0.0, 'min': 0.0, 'max': 0.0}


In [5]:
# Per-face directional breakdown on an east face (should see building in Case A; blocked in Case B)
orient_masks_clear = face_orientation_masks(mesh_clear)
counts_clear, total_clear = face_directional_breakdown(
    mesh_clear, vox_clear, meshsize, orient_masks_clear["+X"], N_azimuth=36, N_elevation=9, offset_vox=0.51
)

orient_masks_trees = face_orientation_masks(mesh_trees)
counts_trees, total_trees = face_directional_breakdown(
    mesh_trees, vox_trees, meshsize, orient_masks_trees["+X"], N_azimuth=36, N_elevation=9, offset_vox=0.51
)

print("Breakdown Case A:", counts_clear)
print("Breakdown Case B:", counts_trees)


Face idx 34, normal [1. 0. 0.], total valid rays: 164
  air_exit: 75 (0.457)
  hit_building: 89 (0.543)
Face idx 34, normal [1. 0. 0.], total valid rays: 164
  air_exit: 162 (0.988)
  tree_block: 2 (0.012)
Breakdown Case A: Counter({'hit_building': 89, 'air_exit': 75})
Breakdown Case B: Counter({'air_exit': 162, 'tree_block': 2})


In [6]:
# Sky view factor sanity checks (target sky: (0,), exclusion mode False)
from voxcity.simulator.view import get_surface_view_factor as get_vf

mesh_clear_sky = get_vf(
    vox_clear,
    meshsize,
    target_values=(0,),
    inclusion_mode=False,  # sky
    N_azimuth=36,
    N_elevation=9,
)
mesh_trees_sky = get_vf(
    vox_trees,
    meshsize,
    target_values=(0,),
    inclusion_mode=False,
    N_azimuth=36,
    N_elevation=9,
)

print_orientation_summary("Sky VF — Case A (clear)", mesh_clear_sky)
print_orientation_summary("Sky VF — Case B (tree wall)", mesh_trees_sky)


Sky VF — Case A (clear)
  +X: {'count': 12, 'mean': 0.3278867102396514, 'min': 0.24509803921568626, 'max': 0.4117647058823529}
  -X: {'count': 12, 'mean': 0.5294117647058824, 'min': 0.5294117647058824, 'max': 0.5294117647058824}
  +Y: {'count': 12, 'mean': 0.5326797385620915, 'min': 0.5326797385620915, 'max': 0.5326797385620915}
  -Y: {'count': 12, 'mean': 0.5508196721311477, 'min': 0.5508196721311476, 'max': 0.5508196721311476}
  +Z: {'count': 8, 'mean': 1.0, 'min': 1.0, 'max': 1.0}
  -Z: {'count': 8, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
Sky VF — Case B (tree wall)
  +X: {'count': 12, 'mean': 0.2188399945789176, 'min': 0.0924451302137817, 'max': 0.40607921762118754}
  -X: {'count': 12, 'mean': 0.5294117647058824, 'min': 0.5294117647058824, 'max': 0.5294117647058824}
  +Y: {'count': 12, 'mean': 0.5326797385620915, 'min': 0.5326797385620915, 'max': 0.5326797385620915}
  -Y: {'count': 12, 'mean': 0.5508196721311477, 'min': 0.5508196721311476, 'max': 0.5508196721311476}
  +Z: {'count': 8,

In [7]:
# Boundary-face NaN test: place building touching x=0 boundary and ensure vertical faces on boundary are NaN
vox_boundary = np.zeros((6, 6, 4), dtype=np.int32)
vox_boundary[0:2, 2:4, 0:2] = -3  # touches x=0 plane
mesh_boundary = get_vf(
    vox_boundary,
    meshsize,
    target_values=(0,),
    inclusion_mode=False,
    N_azimuth=24,
    N_elevation=6,
)
vals = mesh_boundary.metadata["view_factor_values"]
centers = mesh_boundary.triangles_center
normals = mesh_boundary.face_normals

on_boundary = np.isclose(centers[:, 0], 0.0, atol=meshsize*0.05)
vertical = np.abs(normals[:, 2]) < 0.01
mask = on_boundary & vertical
print("Boundary vertical faces — count, NaN count:", int(mask.sum()), int(np.isnan(vals[mask]).sum()))


Boundary vertical faces — count, NaN count: 8 8


In [8]:
# Offset sensitivity test: small vs larger offsets to ensure stability
from voxcity.simulator.view import compute_view_factor_for_all_faces, create_voxel_mesh

bmesh = create_voxel_mesh(vox_clear, -3, meshsize, mesh_type='open_air')
hemi = make_hemisphere_dirs(36, 9)

# Compute both with the same mesh, only changing the ray-origin offset
face_vf_small = compute_view_factor_for_all_faces(
    bmesh.triangles_center,
    bmesh.face_normals,
    hemi,
    vox_clear,
    meshsize,
    0.6,
    1.0,
    (-3,),
    True,
    np.array([[0,0,0],[*vox_clear.shape]], dtype=np.float64) * meshsize,
    meshsize * 0.05,
    True,
    0.51
)

face_vf_large = compute_view_factor_for_all_faces(
    bmesh.triangles_center,
    bmesh.face_normals,
    hemi,
    vox_clear,
    meshsize,
    0.6,
    1.0,
    (-3,),
    True,
    np.array([[0,0,0],[*vox_clear.shape]], dtype=np.float64) * meshsize,
    meshsize * 0.05,
    True,
    1.01
)

# Compare stats on east faces
masks = face_orientation_masks(bmesh)
sel = masks["+X"]
vals_small = face_vf_small[sel]
vals_large = face_vf_large[sel]
print("Offset sensitivity (east faces)")
print({
    "small_offset": {
        "mean": float(np.nanmean(vals_small)),
        "min": float(np.nanmin(vals_small)),
        "max": float(np.nanmax(vals_small)),
    },
    "large_offset": {
        "mean": float(np.nanmean(vals_large)),
        "min": float(np.nanmin(vals_large)),
        "max": float(np.nanmax(vals_large)),
    }
})


Offset sensitivity (east faces)
{'small_offset': {'mean': 0.20806100217864923, 'min': 0.12418300653594772, 'max': 0.2908496732026144}, 'large_offset': {'mean': 0.22930283224400871, 'min': 0.12745098039215685, 'max': 0.32679738562091504}}


In [2]:
# Water view factor validation: building surrounded by large horizontal water surface
# Expectation: For vertical faces, roughly half of outward rays go downward and hit water => VF ~ 0.5

import numpy as np
import math
from voxcity.simulator.view import create_voxel_mesh, compute_view_factor_for_all_faces

# Make this cell self-contained
meshsize = 1.0

def make_hemisphere_dirs(N_azimuth=72, N_elevation=12):
    azimuth_angles = np.linspace(0, 2*np.pi, N_azimuth, endpoint=False)
    elevation_angles = np.linspace(0, np.pi/2, N_elevation)
    hemi = []
    for elev in elevation_angles:
        sin_e = np.sin(elev)
        cos_e = np.cos(elev)
        for az in azimuth_angles:
            x = cos_e * np.cos(az)
            y = cos_e * np.sin(az)
            z = sin_e
            hemi.append([x, y, z])
    return np.array(hemi, dtype=np.float64)

# Scene setup
NX, NY, NZ = 24, 24, 8
vox_water = np.zeros((NX, NY, NZ), dtype=np.int32)

# Water class id (ground water)
WATER_ID = 9

# Fill ground layer with water
vox_water[:, :, 0] = WATER_ID

# Place a box-shaped building in the middle, rising above water
bx0, bx1 = 10, 14
by0, by1 = 10, 14
bz0, bz1 = 0, 4
vox_water[bx0:bx1, by0:by1, bz0:bz1] = -3

# Build the building surface mesh
bmesh_w = create_voxel_mesh(vox_water, -3, meshsize, mesh_type='open_air')

# Hemisphere directions
hemi_w = make_hemisphere_dirs(72, 12)

# Compute water view factor for building faces, including downward rays
vf_water = compute_view_factor_for_all_faces(
    bmesh_w.triangles_center,
    bmesh_w.face_normals,
    hemi_w,
    vox_water,
    meshsize,
    0.6,    # tree_k (unused here)
    1.0,    # tree_lad (unused here)
    (WATER_ID,),
    True,   # inclusion mode: count water hits
    np.array([[0,0,0],[*vox_water.shape]], dtype=np.float64) * meshsize,
    meshsize * 0.05,
    False,  # ignore_downward -> False (consider all outward rays)
    0.51
)

# Summaries for vertical faces
centers_w = bmesh_w.triangles_center
normals_w = bmesh_w.face_normals
is_vertical = np.abs(normals_w[:, 2]) < 1e-6

# All vertical faces
vals_vert = vf_water[is_vertical]
print("Water VF — all vertical faces:", {
    "count": int(vals_vert.size),
    "mean": float(np.nanmean(vals_vert)),
    "min": float(np.nanmin(vals_vert)),
    "max": float(np.nanmax(vals_vert)),
})

# By orientation: compute masks inline
eps = 1e-6
labels = {
    "+X": np.abs(normals_w[:, 0] - 1.0) < eps,
    "-X": np.abs(normals_w[:, 0] + 1.0) < eps,
    "+Y": np.abs(normals_w[:, 1] - 1.0) < eps,
    "-Y": np.abs(normals_w[:, 1] + 1.0) < eps,
}
for label, m in labels.items():
    vals = vf_water[m]
    if vals.size:
        print(f"Water VF — {label}:", {
            "count": int(vals.size),
            "mean": float(np.nanmean(vals)),
            "min": float(np.nanmin(vals)),
            "max": float(np.nanmax(vals)),
        })

# Sanity check: mean should be around 0.5; tolerances depend on sampling
if vals_vert.size:
    approx = float(np.nanmean(vals_vert))
    print("Approx vertical water VF:", approx)
    if 0.35 <= approx <= 0.65:
        print("OK: within expected range ~0.5")
    else:
        print("Note: outside 0.35–0.65; consider increasing sampling or domain size.")


Water VF — all vertical faces: {'count': 96, 'mean': 0.35940948612839213, 'min': 0.2946859903381642, 'max': 0.4316807738814994}
Water VF — +X: {'count': 24, 'mean': 0.3548208534621578, 'min': 0.2946859903381642, 'max': 0.428743961352657}
Water VF — -X: {'count': 24, 'mean': 0.3637781803542673, 'min': 0.30193236714975846, 'max': 0.4311594202898551}
Water VF — +Y: {'count': 24, 'mean': 0.3548208534621578, 'min': 0.2946859903381642, 'max': 0.428743961352657}
Water VF — -Y: {'count': 24, 'mean': 0.36421805723498585, 'min': 0.3022974607013301, 'max': 0.4316807738814994}
Approx vertical water VF: 0.35940948612839213
OK: within expected range ~0.5


In [9]:
import numpy as np
# Water view factor: scene builder and validator
# We will use class id 9 for water voxels in this synthetic test.

WATER_IDS = (9,)

def build_water_scene(config: str):
    # Domain
    NX, NY, NZ = 16, 10, 6
    vox = np.zeros((NX, NY, NZ), dtype=np.int32)
    # Buildings (left and right)
    vox[2:5, 3:7, 0:4] = -3   # left building
    vox[11:14, 3:7, 0:4] = -3 # right building
    # Water channel between x=6..9 spanning same vertical as buildings
    vox[6:10, 3:7, 0:2] = WATER_IDS[0]

    if config == 'clear':
        # no extra occluders
        pass
    elif config == 'blocked_slab':
        # Add a blocking slab in front of left building toward water
        vox[5:6, 3:7, 0:4] = -3
    elif config == 'narrow_gap':
        # Slightly reduce water width
        vox[7:9, 3:7, 0:2] = WATER_IDS[0]
        vox[6:7, 3:7, 0:2] = 0
        vox[9:10, 3:7, 0:2] = 0
    else:
        raise ValueError(f"Unknown config: {config}")
    return vox


def water_vf(meshsize=1.0, config='clear'):
    vox = build_water_scene(config)
    mesh = get_surface_view_factor(
        vox, meshsize,
        target_values=WATER_IDS,
        inclusion_mode=True,   # look for water hits
        N_azimuth=48,
        N_elevation=12,
        progress_report=False,
        adjacent_boundary_classes=(0, -2, 9),  # include water-adjacent building faces
    )
    # Summarize east faces of left building
    centers = mesh.triangles_center
    normals = mesh.face_normals
    vf = mesh.metadata['view_factor_values']
    mask_left = centers[:, 0] < 5 * meshsize
    mask_east = np.abs(normals[:, 0] - 1.0) < 1e-8
    sel = mask_left & mask_east
    vals = vf[sel]
    vals = vals[~np.isnan(vals)]
    stats = {
        'count': int(vals.size),
        'mean': float(np.mean(vals)) if vals.size else np.nan,
        'min': float(np.min(vals)) if vals.size else np.nan,
        'max': float(np.max(vals)) if vals.size else np.nan,
    }
    return vox, mesh, stats

# Run two configs
for cfg in ['clear', 'blocked_slab']:
    vox_w, mesh_w, stats = water_vf(1.0, cfg)
    print(f"Water VF — {cfg}: east faces of left building: {stats}")


Water VF — clear: east faces of left building: {'count': 0, 'mean': nan, 'min': nan, 'max': nan}
Water VF — blocked_slab: east faces of left building: {'count': 0, 'mean': nan, 'min': nan, 'max': nan}


In [ ]:
# Water view factor validation
# Water voxels are class 9 (per OSM mapping). We'll construct scenes:
#  - Clear LoS water channel between buildings
#  - Water occluded by building
#  - Water behind a tree wall (attenuation should block inclusion hits)

WATER = 9
BUILDING = -3
TREE = -2

NX, NY, NZ = 18, 10, 5


def build_water_scene(config: str):
    vox = np.zeros((NX, NY, NZ), dtype=np.int32)
    # Two low buildings left/right
    vox[2:5, 3:7, 0:2] = BUILDING
    vox[12:15, 3:7, 0:2] = BUILDING
    # Water strip in the middle at ground level
    vox[7:11, 3:7, 0:1] = WATER

    if config == 'clear':
        pass
    elif config == 'blocked_by_building':
        # Add a blocker building slab in front of left building to occlude its east faces to water
        vox[6:7, 3:7, 0:2] = BUILDING
    elif config == 'tree_wall':
        # Tree wall in front of left building
        vox[6:7, 3:7, 0:3] = TREE
    else:
        raise ValueError('Unknown config')
    return vox


def water_vf(meshsize=1.0, config='clear'):
    vox = build_water_scene(config)
    mesh = get_surface_view_factor(
        vox,
        meshsize,
        target_values=(WATER,),
        inclusion_mode=True,   # look for water hits
        N_azimuth=36,
        N_elevation=9,
    )
    # summarize +X faces (east faces) of left building
    centers = mesh.triangles_center
    normals = mesh.face_normals
    vf = mesh.metadata['view_factor_values']
    sel_left = centers[:, 0] < 6.0
    east = np.abs(normals[:, 0] - 1.0) < 1e-8
    mask = sel_left & east
    vals = vf[mask]
    vals = vals[~np.isnan(vals)]
    return vox, mesh, {
        'count': int(vals.size),
        'mean': float(np.mean(vals)) if vals.size else np.nan,
        'min': float(np.min(vals)) if vals.size else np.nan,
        'max': float(np.max(vals)) if vals.size else np.nan,
    }

for cfg in ['clear', 'blocked_by_building', 'tree_wall']:
    _, mesh_w, stats = water_vf(1.0, cfg)
    print(f"Water VF ({cfg}) — left +X faces:", stats)


Water VF (clear) — left +X faces: {'count': 16, 'mean': 0.08067810457516339, 'min': 0.0, 'max': 0.19607843137254902}
Water VF (blocked_by_building) — left +X faces: {'count': 16, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
Water VF (tree_wall) — left +X faces: {'count': 16, 'mean': 0.08067810457516339, 'min': 0.0, 'max': 0.19607843137254902}


In [10]:
# Detailed per-ray classification for water VF on one east face
vox_w = build_water_scene('clear')
mesh_w = get_surface_view_factor(
    vox_w, meshsize,
    target_values=WATER_IDS,
    inclusion_mode=True,
    N_azimuth=36,
    N_elevation=9,
    adjacent_boundary_classes=(0, -2, 9),
)

masks = face_orientation_masks(mesh_w)
sel = masks['+X']
if not np.any(sel):
    print("No faces selected.")
else:
    counts, total = face_directional_breakdown(
        mesh_w, vox_w, meshsize, sel, N_azimuth=36, N_elevation=9, offset_vox=0.51
    )
    print("Water VF per-ray breakdown (clear):", counts)


No faces selected.


In [11]:
# Water view factor validation
# Water voxels are class 9 (per OSM mapping). We'll construct scenes:
#  - Clear LoS water channel between buildings
#  - Water occluded by building
#  - Water behind a tree wall (attenuation should block inclusion hits)

WATER = 9
BUILDING = -3
TREE = -2

NX, NY, NZ = 18, 10, 5


def build_water_scene(config: str):
    vox = np.zeros((NX, NY, NZ), dtype=np.int32)
    # Two low buildings left/right
    vox[2:5, 3:7, 0:2] = BUILDING
    vox[12:15, 3:7, 0:2] = BUILDING
    # Water strip in the middle at ground level
    vox[7:11, 3:7, 0:1] = WATER

    if config == 'clear':
        pass
    elif config == 'blocked_by_building':
        # Add a blocker building slab in front of left building to occlude its east faces to water
        vox[6:7, 3:7, 0:2] = BUILDING
    elif config == 'tree_wall':
        # Tree wall in front of left building
        vox[6:7, 3:7, 0:3] = TREE
    else:
        raise ValueError('Unknown config')
    return vox


def water_vf(meshsize=1.0, config='clear'):
    vox = build_water_scene(config)
    mesh = get_surface_view_factor(
        vox,
        meshsize,
        target_values=(WATER,),
        inclusion_mode=True,   # look for water hits
        N_azimuth=36,
        N_elevation=9,
    )
    # summarize +X faces (east faces) of left building
    centers = mesh.triangles_center
    normals = mesh.face_normals
    vf = mesh.metadata['view_factor_values']
    sel_left = centers[:, 0] < 6.0
    east = np.abs(normals[:, 0] - 1.0) < 1e-8
    mask = sel_left & east
    vals = vf[mask]
    vals = vals[~np.isnan(vals)]
    return vox, mesh, {
        'count': int(vals.size),
        'mean': float(np.mean(vals)) if vals.size else np.nan,
        'min': float(np.min(vals)) if vals.size else np.nan,
        'max': float(np.max(vals)) if vals.size else np.nan,
    }

for cfg in ['clear', 'blocked_by_building', 'tree_wall']:
    _, mesh_w, stats = water_vf(1.0, cfg)
    print(f"Water VF ({cfg}) — left +X faces:", stats)


Water VF (clear) — left +X faces: {'count': 16, 'mean': 0.08067810457516339, 'min': 0.0, 'max': 0.19607843137254902}
Water VF (blocked_by_building) — left +X faces: {'count': 16, 'mean': 0.0, 'min': 0.0, 'max': 0.0}
Water VF (tree_wall) — left +X faces: {'count': 16, 'mean': 0.08067810457516339, 'min': 0.0, 'max': 0.19607843137254902}


In [ ]:


centers = mesh.triangles_center
normals = mesh.face_normals
vf = mesh.metadata['view_factor_values']

mask_east = np.isclose(normals[:, 0], 1.0)
mask_left = np.isclose(centers[:, 0], 5*meshsize, atol=meshsize*0.6)  # include plane at x=5
sel = mask_left & mask_east

vals = vf[sel]
vals = vals[~np.isnan(vals)]

NameError: name 'mesh' is not defined